# 模型构建

## 读取特征变换后的数据 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:
train=pd.read_csv('train_FE.csv',index_col=0)
train

,class_id,compartment,level_id,displacement,price,power,cylinder_number,engine_torque,car_height,total_quality,...,d2_mon_201701,d2_mon_201702,d2_mon_201703,d2_mon_201704,d2_mon_201705,d2_mon_201706,d2_mon_201707,d2_mon_201708,d2_mon_201709,d2_mon_201710
0,103507,2,3,1.6,2.846307,6.482565,4,7.493116,13.553132,13.907014,...,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
1,103507,2,3,1.6,2.846307,6.981564,4,8.604063,13.553132,13.990324,...,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0
2,103507,2,3,1.6,2.885846,6.482565,4,7.493116,13.553132,13.716698,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,103507,2,3,1.6,2.885846,6.482565,4,7.493116,13.553132,13.907014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,103507,2,3,1.6,2.885846,6.482565,4,7.493116,13.553132,13.907014,...,0.0,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10217,978089,3,1,1.5,2.736065,6.221214,4,7.338607,13.242388,13.543822,...,0.0,0.0,0.0,205.0,0.0,130.0,-8.0,0.0,-390.0,246.0
10218,978089,3,1,1.5,2.736065,6.221214,4,7.338607,13.242388,13.571680,...,0.0,0.0,0.0,0.0,0.0,0.0,-298.0,0.0,0.0,0.0
10219,978089,3,1,1.5,2.736065,6.221214,4,7.338607,13.242388,13.571680,...,0.0,0.0,0.0,186.0,0.0,167.0,0.0,0.0,-410.0,140.0
10220,978089,3,1,1.5,2.832918,6.221214,4,7.338607,13.242388,13.543822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10222 entries, 0 to 10221
Columns: 612 entries, class_id to d2_mon_201710
dtypes: float64(319), int64(293)
memory usage: 47.8 MB


## 划分特征和标签、训练集和测试集 

考虑用波动程度近似的15年和16年做标签来进行训练，预测17年  
加入一列标签所在年份的特征  
B 榜 201712 的数据预测难度相对更大，因为有 201612 这个异常的前例  
所以我们主打 A 榜 201711  
用 201511、201611 的销量做标签，直接在训练集上做五折交叉验证调参  
直接使用这个模型完成不同月份的预测  
把 201510、201610 的销量做标签，预测 201710 的销量，进行本地验证  
把 201511、201611 的销量做标签，预测 201711 的销量，提交 A 榜  
把 201512、201612 的销量做标签，预测 201712 的销量，提交 B 榜  
过程中需要注意的是，删除训练标签相关的差分特征  
最后的预测结果需要按照 class_id 分组，对标签做 sum 聚合，还要记得把sale_date补充上，按提交要求来  

11月的

In [5]:
train11=train.copy()

In [6]:
cols=['d1_201511','d1_201512','d2_201511','d2_201512','d2_201601','d1_mon_201511','d1_mon_201611','d2_mon_201511','d2_mon_201611']
train11=train11.drop(cols,axis=1)

In [7]:
y_train=pd.concat([train11['sale_date_oh_201511'].copy(),train11['sale_date_oh_201611'].copy()],ignore_index=True)
y_train.shape

(20444,)

In [8]:
train11=train11.drop(['class_id','sale_date_oh_201511','sale_date_oh_201611'],axis=1)
train11.shape

(10222, 600)

In [9]:
train11_cp=train11.copy()
X_test=train11.copy()
train11['year']=15
train11_cp['year']=16
X_train=pd.concat([train11,train11_cp],ignore_index=True)
X_test['year']=17
X_train.shape,X_test.shape

((20444, 601), (10222, 601))

12月的

In [10]:
train12=train.copy()

In [11]:
cols=['d1_201512','d1_201601','d2_201512','d2_201601','d2_201602','d1_mon_201512','d1_mon_201612','d2_mon_201512','d2_mon_201612']
train12=train12.drop(cols,axis=1)

In [12]:
y_train_12=pd.concat([train12['sale_date_oh_201512'].copy(),train12['sale_date_oh_201612'].copy()],ignore_index=True)
y_train_12.shape

(20444,)

In [13]:
train12=train12.drop(['class_id','sale_date_oh_201512','sale_date_oh_201612'],axis=1)
train12.shape

(10222, 600)

In [14]:
train12_cp=train12.copy()
X_test_12=train12.copy()
train12['year']=15
train12_cp['year']=16
X_train_12=pd.concat([train12,train12_cp],ignore_index=True)
X_test_12['year']=17
X_train_12.shape,X_test_12.shape

((20444, 601), (10222, 601))

10月的

In [15]:
train10=train.copy()

In [16]:
cols=['d1_201510','d1_201511','d2_201510','d2_201511','d2_201512','d1_mon_201510',
      'd1_mon_201610','d2_mon_201510','d2_mon_201610','d2_mon_201710']
train10=train10.drop(cols,axis=1)

In [17]:
y_train_10=pd.concat([train10['sale_date_oh_201510'].copy(),train10['sale_date_oh_201610'].copy()],ignore_index=True)
y_test_10=train10['sale_date_oh_201710'].copy()
y_train_10.shape,y_test_10.shape

((20444,), (10222,))

In [18]:
train10=train10.drop(['class_id','sale_date_oh_201510','sale_date_oh_201610','sale_date_oh_201710'],axis=1)
train10.shape

(10222, 598)

In [19]:
train10_cp=train10.copy()
X_test_10=train10.copy()
train10['year']=15
train10_cp['year']=16
X_train_10=pd.concat([train10,train10_cp],ignore_index=True)
X_test_10['year']=17
X_train_10.shape,X_test_10.shape

((20444, 599), (10222, 599))

## 定义五折交叉验证函数

In [20]:
def rmse_cv(model):
    rmse=-cross_val_score(model,X_train,y_train,scoring='neg_root_mean_squared_error',cv=5)
    return rmse.mean(),rmse.std()

## 默认超参数观察所有选定模型的效果

In [21]:
# 线性回归
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
rmse_cv(reg)

(17.584506521049658, 3.421066840362865)

In [22]:
# 岭回归
from sklearn.linear_model import RidgeCV
reg=RidgeCV()
rmse_cv(reg)

(17.551881228361186, 3.4403297625966203)

In [23]:
# 随机梯度下降
from sklearn.linear_model import SGDRegressor
reg=SGDRegressor()
rmse_cv(reg)

(109488013778983.84, 31270066348206.465)

In [24]:
# lasso回归
from sklearn.linear_model import LassoCV
reg=LassoCV()
rmse_cv(reg)

(18.80204511526112, 3.317053686206642)

In [25]:
# 弹性网络
from sklearn.linear_model import ElasticNetCV
reg=ElasticNetCV()
rmse_cv(reg)

(18.785215158956497, 3.2432966536221657)

In [26]:
# svm
from sklearn.svm import SVR
reg=SVR()
rmse_cv(reg)

(19.127340387405674, 2.1311315351948568)

In [27]:
# 决策树
from sklearn.tree import DecisionTreeRegressor
reg=DecisionTreeRegressor()
rmse_cv(reg)

(16.979928581916244, 6.672636495245062)

In [28]:
# 极限树
from sklearn.tree import ExtraTreeRegressor
reg=ExtraTreeRegressor()
rmse_cv(reg)

(16.898019384284325, 9.925395970099311)

In [29]:
# adaboost
from sklearn.ensemble import AdaBoostRegressor
reg=AdaBoostRegressor()
rmse_cv(reg)

(25.14745564585075, 15.182527383358874)

In [30]:
# bagging
from sklearn.ensemble import BaggingRegressor
reg=BaggingRegressor()
rmse_cv(reg)

(13.761124246078335, 5.889045919121722)

In [31]:
# 极限随机树
from sklearn.ensemble import ExtraTreesRegressor
reg=ExtraTreesRegressor()
rmse_cv(reg)

(12.297424119698688, 6.363977145815164)

In [32]:
# 梯度提升决策树
from sklearn.ensemble import GradientBoostingRegressor
reg=GradientBoostingRegressor()
rmse_cv(reg)

(12.165384397507188, 5.289106798882194)

In [33]:
# 随机森林
from sklearn.ensemble import RandomForestRegressor
reg=RandomForestRegressor()
rmse_cv(reg)

(12.881916182578674, 5.62271773690681)

In [34]:
# knn
from sklearn.neighbors import KNeighborsRegressor
reg=KNeighborsRegressor()
rmse_cv(reg)

(20.341516999269064, 1.224284402066326)

In [35]:
# 限定半径最近邻回归树
# from sklearn.neighbors import RadiusNeighborsRegressor
# reg=RadiusNeighborsRegressor()
# rmse_cv(reg)
# UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
# 这个不行

In [36]:
# xgboost
from xgboost import XGBRegressor
reg=XGBRegressor()
rmse_cv(reg)

(14.07841639142581, 3.399830167608544)

In [37]:
# xgboost随机森林
from xgboost import XGBRFRegressor
reg=XGBRFRegressor()
rmse_cv(reg)

(12.160439068995938, 5.184023958419302)

In [38]:
# lgbm
from lightgbm import LGBMRegressor
reg=LGBMRegressor()
rmse_cv(reg)

(12.122678184394516, 5.10419514443187)

通过结果可以看到，树形模型效果都挺不错的  
其中rmse五折均值最小的三个模型是 lgbm, xgbrf, gbdt  
考虑对这三个模型进一步调参，使用懒人stacking融合  

## 调参

调参使用的是网格搜索+手动玄学瞎试  
本文档省略了调参过程的记录，仅保留最终的结果

In [39]:
from lightgbm import LGBMRegressor
lgbm=LGBMRegressor(learning_rate=0.03,num_leaves=63)
rmse_cv(lgbm)

(11.27050850184436, 5.139872709261197)

In [40]:
# xgbrf
from xgboost import XGBRFRegressor
xgbrf=XGBRFRegressor(random_state=0,subsample=0.5,max_depth=5,min_child_weight=1.2)
rmse_cv(xgbrf)

(11.61352752585535, 5.720931179398938)

In [41]:
# gbdt
from sklearn.ensemble import GradientBoostingRegressor
gbdt=GradientBoostingRegressor(random_state=0)
rmse_cv(reg)
# 不管咋调，最好都只有十二点一几的rmse，只会变得更差，不会有明显变好，太离谱了，最后我们保留了这个默认参数的GBDT

(12.122678184394516, 5.10419514443187)

## 模型融合

懒人融合法之stacking

In [42]:
from sklearn.ensemble import StackingRegressor
stacking=StackingRegressor([('lgbm',lgbm),('xgbrf',xgbrf),('gbdt',gbdt)],final_estimator=lgbm,cv=5)
rmse_cv(stacking)

(11.984012218202823, 5.092967833654622)

看起来融合之后似乎比单独的调参后的 lgbm 差了一点点  
关于这个问题我之前也简单查过，给出的答案是，理论上来讲，stacking应该不会明显变差  
实际上我认为这样融合之后，应该在一定程度上消除了一点过拟合  

## 本地验证 

In [43]:
from sklearn.metrics import mean_squared_error
stacking.fit(X_train_10,y_train_10)
y_pred_10=stacking.predict(X_test_10)
y_test_tmp=pd.DataFrame({'class_id':train['class_id'].tolist(),'sale_quantity':y_test_10.tolist()})
y_pred_tmp=pd.DataFrame({'class_id':train['class_id'].tolist(),'sale_quantity':y_pred_10.tolist()})
y_test_group=y_test_tmp.groupby('class_id').sum()
y_pred_group=y_pred_tmp.groupby('class_id').sum()
np.sqrt(mean_squared_error(y_test_group,y_pred_group))

173.05585384701337

In [44]:
y_test_group

,sale_quantity
class_id,
103507,180.0
124140,263.0
125403,156.0
136916,162.0
169673,152.0
...,...
961362,73.0
961962,80.0
963845,207.0


In [45]:
y_pred_group

,sale_quantity
class_id,
103507,126.097782
124140,314.349413
125403,134.062990
136916,186.732947
169673,0.253726
...,...
961362,63.129891
961962,216.200524
963845,247.041849


简单看了一下本地验证的效果  
感觉还不错，大部分都是差个10到60那样  
rmse 173，很不错  
10月总体销售量会比11月小一点  
所以11、12月的 rmse 肯定还要高一些  
但差距不会太大，毕竟预测的手法上是完全一致的，评分方式也准确无误  
本地验证的结果完全可以用来做参考的  
我认为我们的结果至少打进复赛是没啥问题的  

## 对天池测试集预测并保存为CSV

In [46]:
stacking.fit(X_train,y_train)
y_predA=stacking.predict(X_test)
y_predA_tmp=pd.DataFrame({'class_id':train['class_id'].tolist(),'predict_quantity':y_predA.tolist()})
y_predA_group=y_predA_tmp.groupby('class_id').sum()

In [47]:
y_predA_group.to_csv('submission_A.csv')
df=pd.read_csv('submission_A.csv')
df.insert(0,'predict_date',201711)
df.to_csv('submission_A.csv',index=0)
df

,predict_date,class_id,predict_quantity
0,201711,103507,80.667825
1,201711,124140,325.773384
2,201711,125403,115.012354
3,201711,136916,249.757774
4,201711,169673,0.318698
...,...,...,...
135,201711,961362,80.122599
136,201711,961962,249.523367
137,201711,963845,174.649047
138,201711,973106,12.637976


In [48]:
stacking.fit(X_train_12,y_train_12)
y_predB=stacking.predict(X_test_12)
y_predB_tmp=pd.DataFrame({'class_id':train['class_id'].tolist(),'predict_quantity':y_predB.tolist()})
y_predB_group=y_predB_tmp.groupby('class_id').sum()

In [49]:
y_predB_group.to_csv('submission_B.csv')
df=pd.read_csv('submission_B.csv')
df.insert(0,'predict_date',201712)
df.to_csv('submission_B.csv',index=0)
df

,predict_date,class_id,predict_quantity
0,201712,103507,641.770161
1,201712,124140,519.520347
2,201712,125403,198.829271
3,201712,136916,345.108314
4,201712,169673,0.495891
...,...,...,...
135,201712,961362,141.844900
136,201712,961962,347.268192
137,201712,963845,274.287513
138,201712,973106,22.744881


完结撒花~